## Text Classification

fastText is a library for efficient learning of word representations and sentence classification.

The goal of text classification is to assign documents (such as emails, posts, text messages, product reviews, etc...) to one or multiple categories. Such categories can be review scores, spam v.s. non-spam, or the language in which the document was typed. Nowadays, the dominant approach to build such classifiers is machine learning, that is learning classification rules from examples. In order to build such classifiers, we need labeled data, which consists of documents and their corresponding categories (or tags, or labels).

As an example, we build a classifier which automatically classifies stackexchange questions about cooking into one of several possible tags, such as `pot`, `bowl` or `baking`.

This example notebook is base on the guide from fasttext: https://fasttext.cc/docs/en/supervised-tutorial.html

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [3]:
!pip install fasttext
!pip install bentoml

### Prepare data

In [8]:
!curl https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz --output cooking.stackexchange.tar.gz && tar xvzf cooking.stackexchange.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  446k  100  446k    0     0   847k      0 --:--:-- --:--:-- --:--:--  849k
x cooking.stackexchange.id
x cooking.stackexchange.txt
x readme.txt


In [9]:
!head cooking.stackexchange.txt

__label__sauce __label__cheese How much does potato starch affect a cheese sauce recipe?
__label__food-safety __label__acidity Dangerous pathogens capable of growing in acidic environments
__label__cast-iron __label__stove How do I cover up the white spots on my cast iron stove?
__label__restaurant Michelin Three Star Restaurant; but if the chef is not there
__label__knife-skills __label__dicing Without knife skills, how can I quickly and accurately dice vegetables?
__label__storage-method __label__equipment __label__bread What's the purpose of a bread box?
__label__baking __label__food-safety __label__substitutions __label__peanuts how to seperate peanut oil from roasted peanuts at home?
__label__chocolate American equivalent for British chocolate terms
__label__baking __label__oven __label__convection Fan bake vs bake
__label__sauce __label__storage-lifetime __label__acidity __label__mayonnaise Regulation and balancing of readymade packed mayonnaise and other sauces


In [10]:
!head -n 12404 cooking.stackexchange.txt > cooking.train
!tail -n 3000 cooking.stackexchange.txt > cooking.valid

### Train model

In [2]:
import fasttext

In [3]:
model = fasttext.train_supervised(input="cooking.train")

### Define and save BentoService

In [9]:
%%writefile text_classification.py

from bentoml import env, artifacts, BentoService, api
from bentoml.artifact import FasttextModelArtifact
from bentoml.handlers import JsonHandler

@env(auto_pip_dependencies=True)
@artifacts([FasttextModelArtifact('model')])
class FasttextClassification(BentoService):
    
    @api(JsonHandler)
    def predict(self, parsed_json):
        result = self.artifacts.model.predict(parsed_json['text'])
        # return top result
        return result[0][0].replace('__label__', '')

Writing text_classification.py


In [10]:
from text_classification import FasttextClassification

svc = FasttextClassification()
svc.pack('model', model)

saved_path = svc.save()

[2020-05-07 13:25:36,186] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.7.4. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-07 13:25:36,626] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.7.4. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml

no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check
creating BentoML-0.7.4+8.g496f22a
creating BentoML-0.7.4+8.g496f22a/BentoML.egg-info
creating BentoML-0.7.4+8.g496f22a/bentoml
creating BentoML-0.7.4+8.g496f22a/bentoml/artifact
creating BentoML-0.7.4+8.g496f22a/bentoml/bundler
creating BentoML-0.7.4+8.g496f22a/bentoml/cli
creating BentoML-0.7.4+8.g496f22a/bentoml/clipper
creating BentoML-0.7.4+8.g496f22a/bentoml/configuration
creating BentoML-0.7.4+8.g496f22a/bentoml/configuration/__pycache__
creating BentoML-0.7.4+8.g496f22a/bentoml/deployment
creating BentoML-0.7.4+8.g496f22a/bentoml/deployment/aws_lambda
creating BentoML-0.7.4+8.g496f22a/bentoml/deployment/sagemaker
creating BentoML-0.7.4+8.g496f22a/bentoml/handlers
creating BentoML-0.7.4+8.g496f22a/bentoml/marshal
creating BentoML-0.7.4+8.g496f22a/bentoml/migrations
creating BentoML-0.7.4+8.g496f22a/bentoml/migrations/__pycache__
creating BentoML-0.7.4+8.g496f22a/bentoml/migrations/versions
creating BentoML-0.7.4+8

creating BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/bytes
creating BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/camelcase
creating BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/chownr
creating BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/cliui
creating BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/code-point-at
creating BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/colour
creating BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/colour/examples
creating BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/colour/externs
creating BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/colour/tests
creating BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/concat-map
creating BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/concat-map/example
creating BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/concat-map/test
creating BentoML-0.7

creating BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/grpc/deps/grpc/src/core/ext/filters/message_size
creating BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/grpc/deps/grpc/src/core/ext/filters/workarounds
creating BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/grpc/deps/grpc/src/core/ext/transport
creating BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/grpc/deps/grpc/src/core/ext/transport/chttp2
creating BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/grpc/deps/grpc/src/core/ext/transport/chttp2/alpn
creating BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/grpc/deps/grpc/src/core/ext/transport/chttp2/client
creating BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/grpc/deps/grpc/src/core/ext/transport/chttp2/client/insecure
creating BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/grpc/deps/grpc/src/core/ext/transport/chttp2/client/secure
creating BentoML-0.7.4+8.g496f22a/bent

...

copying bentoml/yatai/web/dist/node_modules/readable-stream/LICENSE -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/readable-stream
copying bentoml/yatai/web/dist/node_modules/readable-stream/README.md -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/readable-stream
copying bentoml/yatai/web/dist/node_modules/readable-stream/duplex-browser.js -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/readable-stream
copying bentoml/yatai/web/dist/node_modules/readable-stream/duplex.js -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/readable-stream
copying bentoml/yatai/web/dist/node_modules/readable-stream/package.json -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/readable-stream
copying bentoml/yatai/web/dist/node_modules/readable-stream/passthrough.js -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/readable-stream
copying bentoml/yatai/web/dist/node_modules/readable-stream/readable-browser.js -> Ben

copying bentoml/yatai/web/dist/node_modules/send/node_modules/http-errors/LICENSE -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/send/node_modules/http-errors
copying bentoml/yatai/web/dist/node_modules/send/node_modules/http-errors/README.md -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/send/node_modules/http-errors
copying bentoml/yatai/web/dist/node_modules/send/node_modules/http-errors/index.js -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/send/node_modules/http-errors
copying bentoml/yatai/web/dist/node_modules/send/node_modules/http-errors/package.json -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/send/node_modules/http-errors
copying bentoml/yatai/web/dist/node_modules/send/node_modules/ms/index.js -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/send/node_modules/ms
copying bentoml/yatai/web/dist/node_modules/send/node_modules/ms/license.md -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/no

copying bentoml/yatai/web/dist/node_modules/tar/package.json -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/tar
copying bentoml/yatai/web/dist/node_modules/tar/lib/buffer.js -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/tar/lib
copying bentoml/yatai/web/dist/node_modules/tar/lib/create.js -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/tar/lib
copying bentoml/yatai/web/dist/node_modules/tar/lib/extract.js -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/tar/lib
copying bentoml/yatai/web/dist/node_modules/tar/lib/header.js -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/tar/lib
copying bentoml/yatai/web/dist/node_modules/tar/lib/high-level-opt.js -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/tar/lib
copying bentoml/yatai/web/dist/node_modules/tar/lib/large-numbers.js -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/tar/lib
copying bentoml/yatai/web/dist/node_modules/tar/lib

copying bentoml/yatai/web/dist/node_modules/wide-align/align.js -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/wide-align
copying bentoml/yatai/web/dist/node_modules/wide-align/package.json -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/wide-align
copying bentoml/yatai/web/dist/node_modules/wide-align/node_modules/ansi-regex/index.js -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/wide-align/node_modules/ansi-regex
copying bentoml/yatai/web/dist/node_modules/wide-align/node_modules/ansi-regex/license -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/wide-align/node_modules/ansi-regex
copying bentoml/yatai/web/dist/node_modules/wide-align/node_modules/ansi-regex/package.json -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/wide-align/node_modules/ansi-regex
copying bentoml/yatai/web/dist/node_modules/wide-align/node_modules/ansi-regex/readme.md -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/w

copying bentoml/yatai/web/dist/node_modules/yargs/locales/fr.json -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/yargs/locales
copying bentoml/yatai/web/dist/node_modules/yargs/locales/id.json -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/yargs/locales
copying bentoml/yatai/web/dist/node_modules/yargs/locales/ja.json -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/yargs/locales
copying bentoml/yatai/web/dist/node_modules/yargs/locales/ko.json -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/yargs/locales
copying bentoml/yatai/web/dist/node_modules/yargs/locales/nb.json -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/yargs/locales
copying bentoml/yatai/web/dist/node_modules/yargs/locales/pirate.json -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/yargs/locales
copying bentoml/yatai/web/dist/node_modules/yargs/locales/pl.json -> BentoML-0.7.4+8.g496f22a/bentoml/yatai/web/dist/node_modules/ya

### Use BentoService with BentoML CLI


Use bentoml get to list all versions of BentoService, including the version tag will display additional information and metadata

In [11]:
!bentoml get FasttextClassification


BENTO_SERVICE                                 AGE                           APIS                  ARTIFACTS
FasttextClassification:20200507132536_C23A76  38 minutes and 11.25 seconds  predict<JsonHandler>  model<FasttextModelArtifact>


In [16]:
!bentoml get FasttextClassification:{svc.version}

{
  "name": "FasttextClassification",
  "version": "20200507132536_C23A76",
  "uri": {
    "type": "LOCAL",
    "uri": "/Users/bozhaoyu/bentoml/repository/FasttextClassification/20200507132536_C23A76"
  },
  "bentoServiceMetadata": {
    "name": "FasttextClassification",
    "version": "20200507132536_C23A76",
    "createdAt": "2020-05-07T20:25:53.878686Z",
    "env": {
      "condaEnv": "name: bentoml-FasttextClassification\nchannels:\n- defaults\ndependencies:\n- python=3.7.3\n- pip\n",
      "pipDependencies": "bentoml==0.7.4\nfasttext",
      "pythonVersion": "3.7.3"
    },
    "artifacts": [
      {
        "name": "model",
        "artifactType": "FasttextModelArtifact"
      }
    ],
    "apis": [
      {
        "name": "predict",
        "handlerType": "JsonHandler",
        "docs": "BentoService API",
        "handlerConfig": {
          "is_batch_input": false
        }
      }
    ]
  }
}


In [29]:
!bentoml run FasttextClassification:20200507132536_C23A76 predict --input='{"text": "Which baking dish is best to bake a banana bread ?"}'

[2020-05-07 15:50:49,100] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.7.4. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-07 15:50:49,113] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.7.4,  but loading from BentoML version 0.7.4+8.g496f22a
[2020-05-07 15:50:49,130] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be

### Model Serving via REST API


Note: Running as local rest api server does not work with Google Colab, please copy this notebook to run it locally

In [13]:
!bentoml serve {saved_path}

[2020-05-07 14:06:25,817] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.7.4. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-07 14:06:25,836] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.7.4,  but loading from BentoML version 0.7.4+8.g496f22a
[2020-05-07 14:06:25,849] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be

#### Send prediction requeset to the REST API server

```
curl -X POST \
  http://localhost:5000/predict \
  -H 'Content-Type: application/json' \
  -d '{"text": "Which baking dish is best to bake a banana bread ?"}'
```


### Install saved BentoService as PyPI package

In [14]:
!pip install {saved_path}

Processing /Users/bozhaoyu/bentoml/repository/FasttextClassification/20200507132536_C23A76


  Created wheel for FasttextClassification: filename=FasttextClassification-20200507132536_C23A76-py3-none-any.whl size=5264253 sha256=fb0af0ac967ea780693bf0634893c826ad1e38f29ca170fba008f655adb0e25f
  Stored in directory: /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/pip-ephem-wheel-cache-60d124pk/wheels/c0/7f/c6/0990be9348c32f01dfeb8ad3ebfaab9d3c43498c92ab15ddab
Successfully built FasttextClassification


In [15]:
!FasttextClassification info

[2020-05-07 14:15:11,108] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.7.4. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-07 14:15:11,117] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.7.4,  but loading from BentoML version 0.7.4+8.g496f22a
{
  "name": "FasttextClassification",
  "version": "20200507132536_C23A76",
  "created_at": "2020-05-07T20:25:53.878686Z",
  "env": {
    "conda_env": "name: bentoml-FasttextClassification\nchannels:\n- defaults\ndependencies:\n

In [24]:
!FasttextClassification run predict --input '{"text": "Which baking dish is best to bake a banana bread ?"}'

[2020-05-07 14:59:39,727] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.7.4. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-07 14:59:39,745] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.7.4,  but loading from BentoML version 0.7.4+8.g496f22a
[2020-05-07 14:59:39,747] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be

### Deploy BentoService as REST API server to the cloud


BentoML support deployment to multiply cloud provider services, such as AWS Lambda, AWS Sagemaker, Google Cloudrun and etc. You can find the full list and guide on the documentation site at https://docs.bentoml.org/en/latest/deployment/index.html

For this project, we are going to deploy to AWS Sagemaker

`bentoml sagemaker deploy` command deploys to AWS Sagemaker with a single command

In [19]:
!bentoml sagemaker deploy fasttext-classification -b FasttextClassification:{svc.version} --api-name predict

Deploying Sagemaker deployment /[2020-05-07 14:22:03,650] INFO - Step 1/11 : FROM continuumio/miniconda3:4.7.12
[2020-05-07 14:22:03,650] INFO - 

[2020-05-07 14:22:03,650] INFO -  ---> 406f2b43ea59

[2020-05-07 14:22:03,651] INFO - Step 2/11 : EXPOSE 8080
[2020-05-07 14:22:03,651] INFO - 

\[2020-05-07 14:22:03,784] INFO -  ---> Running in aeef665f0547

-[2020-05-07 14:22:03,892] INFO -  ---> 028063f322d0

[2020-05-07 14:22:03,892] INFO - Step 3/11 : RUN set -x      && apt-get update      && apt-get install --no-install-recommends --no-install-suggests -y libpq-dev build-essential     && apt-get install -y nginx      && rm -rf /var/lib/apt/lists/*
[2020-05-07 14:22:03,892] INFO - 

[2020-05-07 14:22:03,954] INFO -  ---> Running in d3941cbaee4d

-[2020-05-07 14:22:04,306] INFO - + apt-get update

|[2020-05-07 14:22:04,559] INFO - Get:1 http://deb.debian.org/debian buster InRelease [122 kB]

\[2020-05-07 14:22:04,631] INFO - Get:2 http://deb.debian.org/debian buster-updates InRelea

-[2020-05-07 14:22:18,598] INFO - Get:33 http://deb.debian.org/debian buster/main amd64 dpkg-dev all 1.19.7 [1773 kB]

\[2020-05-07 14:22:20,537] INFO - Get:34 http://deb.debian.org/debian buster/main amd64 build-essential amd64 12.6 [7576 B]

/[2020-05-07 14:22:20,778] INFO - debconf: delaying package configuration, since apt-utils is not installed

|[2020-05-07 14:22:20,814] INFO - Fetched 47.2 MB in 11s (4418 kB/s)

[2020-05-07 14:22:20,844] INFO - Selecting previously unselected package binutils-common:amd64.
(Reading database ... 
(Reading database ... 75% INFO - (Reading database ... 5%
[2020-05-07 14:22:20,861] INFO - (Reading database ... 80%
[2020-05-07 14:22:20,880] INFO - (Reading database ... 85%
[2020-05-07 14:22:20,897] INFO - (Reading database ... 90%
\[2020-05-07 14:22:20,916] INFO - (Reading database ... 95%
(Reading database ... 12557 files and directories currently installed.)

[2020-05-07 14:22:20,935] INFO - Preparing to unpack .../00-binutils-common_2.31.1-16

|[2020-05-07 14:22:26,919] INFO - Selecting previously unselected package libdpkg-perl.

[2020-05-07 14:22:26,921] INFO - Preparing to unpack .../29-libdpkg-perl_1.19.7_all.deb ...

[2020-05-07 14:22:26,925] INFO - Unpacking libdpkg-perl (1.19.7) ...

[2020-05-07 14:22:27,012] INFO - Selecting previously unselected package dpkg-dev.

[2020-05-07 14:22:27,015] INFO - Preparing to unpack .../30-dpkg-dev_1.19.7_all.deb ...

\[2020-05-07 14:22:27,018] INFO - Unpacking dpkg-dev (1.19.7) ...

-[2020-05-07 14:22:27,123] INFO - Selecting previously unselected package build-essential.

[2020-05-07 14:22:27,125] INFO - Preparing to unpack .../31-build-essential_12.6_amd64.deb ...

[2020-05-07 14:22:27,130] INFO - Unpacking build-essential (12.6) ...

[2020-05-07 14:22:27,167] INFO - Selecting previously unselected package libpq5:amd64.

[2020-05-07 14:22:27,170] INFO - Preparing to unpack .../32-libpq5_11.7-0+deb10u1_amd64.deb ...

[2020-05-07 14:22:27,174] INFO - Unpacking libpq5:amd64 (11.7

/[2020-05-07 14:22:30,490] INFO - Get:19 http://deb.debian.org/debian buster/main amd64 libnginx-mod-http-geoip amd64 1.14.2-2+deb10u1 [93.8 kB]

[2020-05-07 14:22:30,511] INFO - Get:20 http://deb.debian.org/debian buster/main amd64 libnginx-mod-http-image-filter amd64 1.14.2-2+deb10u1 [97.4 kB]

[2020-05-07 14:22:30,529] INFO - Get:21 http://deb.debian.org/debian buster/main amd64 libnginx-mod-http-subs-filter amd64 1.14.2-2+deb10u1 [95.7 kB]

[2020-05-07 14:22:30,549] INFO - Get:22 http://deb.debian.org/debian buster/main amd64 libnginx-mod-http-upstream-fair amd64 1.14.2-2+deb10u1 [95.7 kB]

[2020-05-07 14:22:30,565] INFO - Get:23 http://deb.debian.org/debian buster/main amd64 libnginx-mod-http-xslt-filter amd64 1.14.2-2+deb10u1 [95.7 kB]

[2020-05-07 14:22:30,581] INFO - Get:24 http://deb.debian.org/debian buster/main amd64 libnginx-mod-mail amd64 1.14.2-2+deb10u1 [126 kB]

|[2020-05-07 14:22:30,605] INFO - Get:25 http://deb.debian.org/debian buster/main amd64 libnginx-mod-stream

-[2020-05-07 14:22:32,864] INFO - Selecting previously unselected package libnginx-mod-mail.

[2020-05-07 14:22:32,865] INFO - Preparing to unpack .../23-libnginx-mod-mail_1.14.2-2+deb10u1_amd64.deb ...

[2020-05-07 14:22:32,869] INFO - Unpacking libnginx-mod-mail (1.14.2-2+deb10u1) ...

[2020-05-07 14:22:32,916] INFO - Selecting previously unselected package libnginx-mod-stream.

[2020-05-07 14:22:32,919] INFO - Preparing to unpack .../24-libnginx-mod-stream_1.14.2-2+deb10u1_amd64.deb ...

/[2020-05-07 14:22:32,924] INFO - Unpacking libnginx-mod-stream (1.14.2-2+deb10u1) ...

[2020-05-07 14:22:32,969] INFO - Selecting previously unselected package nginx-full.

[2020-05-07 14:22:32,971] INFO - Preparing to unpack .../25-nginx-full_1.14.2-2+deb10u1_amd64.deb ...

[2020-05-07 14:22:32,974] INFO - Unpacking nginx-full (1.14.2-2+deb10u1) ...

|[2020-05-07 14:22:33,049] INFO - Selecting previously unselected package nginx.

[2020-05-07 14:22:33,051] INFO - Preparing to unpack .../26-ngin

[2020-05-07 14:22:41,617] INFO - 
numpy-1.18.1         | 5 KB      |            |   0% 
numpy-1.18.1         | 5 KB      | ########## | 100% 
[2020-05-07 14:22:41,653] INFO - 
openssl-1.1.1g       | 2.5 MB    |            |   0% 
openssl-1.1.1g       | 2.5 MB    | #4         |  15% 
openssl-1.1.1g       | 2.5 MB    | ###6       |  36% 
openssl-1.1.1g       | 2.5 MB    | #####7     |  58% 
openssl-1.1.1g       | 2.5 MB    | ########   |  80% 
openssl-1.1.1g       | 2.5 MB    | #########8 |  99% 
openssl-1.1.1g       | 2.5 MB    | ########## | 100% 
[2020-05-07 14:22:42,250] INFO - 
mkl_random-1.1.0     | 321 KB    |            |   0% 
mkl_random-1.1.0     | 321 KB    | ########## | 100% 
[2020-05-07 14:22:42,364] INFO - 
mkl-2020.0           | 128.9 MB  |            |   0% 
mkl-2020.0           | 128.9 MB  |            |   0% 
mkl-2020.0           | 128.9 MB  |            |   1% 
mkl-2020.0           | 128.9 MB  | 1          |   1% 
mkl-2020.0           | 128.9 MB  | 1          |   1% 


mkl-2020.0           | 128.9 MB  | #####7     |  58% 
mkl-2020.0           | 128.9 MB  | #####8     |  58% 
mkl-2020.0           | 128.9 MB  | #####8     |  58% 
mkl-2020.0           | 128.9 MB  | #####8     |  59% 
mkl-2020.0           | 128.9 MB  | #####9     |  59% 
mkl-2020.0           | 128.9 MB  | #####9     |  60% 
mkl-2020.0           | 128.9 MB  | ######     |  60% 
mkl-2020.0           | 128.9 MB  | ######     |  61% 
mkl-2020.0           | 128.9 MB  | ######     |  61% 
mkl-2020.0           | 128.9 MB  | ######1    |  61% 
mkl-2020.0           | 128.9 MB  | ######1    |  62% 
mkl-2020.0           | 128.9 MB  | ######2    |  62% 
mkl-2020.0           | 128.9 MB  | ######2    |  63% 
mkl-2020.0           | 128.9 MB  | ######3    |  63% 
mkl-2020.0           | 128.9 MB  | ######3    |  63% 
mkl-2020.0           | 128.9 MB  | ######3    |  64% 
mkl-2020.0           | 128.9 MB  | ######4    |  64% 
mkl-2020.0           | 128.9 MB  | ######4    |  65% 
mkl-2020.0           | 128.9

numpy-base-1.18.1    | 4.2 MB    | 9          |   9% 
numpy-base-1.18.1    | 4.2 MB    | ##1        |  21% 
numpy-base-1.18.1    | 4.2 MB    | ###4       |  35% 
numpy-base-1.18.1    | 4.2 MB    | ####7      |  47% 
numpy-base-1.18.1    | 4.2 MB    | #####6     |  57% 
numpy-base-1.18.1    | 4.2 MB    | ######7    |  68% 
numpy-base-1.18.1    | 4.2 MB    | ########   |  80% 
numpy-base-1.18.1    | 4.2 MB    | #########2 |  93% 
numpy-base-1.18.1    | 4.2 MB    | ########## | 100% 
[2020-05-07 14:23:17,892] INFO - 
conda-4.8.3          | 2.8 MB    |            |   0% 
conda-4.8.3          | 2.8 MB    | #          |  10% 
conda-4.8.3          | 2.8 MB    | ##8        |  29% 
conda-4.8.3          | 2.8 MB    | ####6      |  47% 
conda-4.8.3          | 2.8 MB    | ######6    |  66% 
conda-4.8.3          | 2.8 MB    | ########4  |  84% 
conda-4.8.3          | 2.8 MB    | ########## | 100% 
[2020-05-07 14:23:18,599] INFO - 
mkl_fft-1.0.15       | 154 KB    |            |   0% 
mkl_fft-1.0.15

\[2020-05-07 14:24:15,817] INFO - Collecting ruamel.yaml>=0.15.0

[2020-05-07 14:24:15,837] INFO -   Downloading ruamel.yaml-0.16.10-py2.py3-none-any.whl (111 kB)

\[2020-05-07 14:24:16,590] INFO - Collecting boto3

[2020-05-07 14:24:16,612] INFO -   Downloading boto3-1.13.5-py2.py3-none-any.whl (128 kB)

-[2020-05-07 14:24:16,682] INFO - Requirement already satisfied: numpy in /opt/conda/lib/python3.7/site-packages (from bentoml==0.7.4->-r /opt/program/requirements.txt (line 1)) (1.18.1)

/[2020-05-07 14:24:16,809] INFO - Collecting humanfriendly

[2020-05-07 14:24:16,832] INFO -   Downloading humanfriendly-8.2-py2.py3-none-any.whl (86 kB)

-[2020-05-07 14:24:17,562] INFO - Collecting aiohttp

/[2020-05-07 14:24:17,589] INFO -   Downloading aiohttp-3.6.2-cp37-cp37m-manylinux1_x86_64.whl (1.2 MB)

\[2020-05-07 14:24:18,284] INFO - Collecting psutil

-[2020-05-07 14:24:18,308] INFO -   Downloading psutil-5.7.0.tar.gz (449 kB)

/[2020-05-07 14:24:18,900] INFO - Requirement alrea

-[2020-05-07 14:24:35,400] INFO - Collecting docutils<0.16,>=0.10

[2020-05-07 14:24:35,419] INFO -   Downloading docutils-0.15.2-py3-none-any.whl (547 kB)

\[2020-05-07 14:24:35,702] INFO - Collecting MarkupSafe>=0.23

[2020-05-07 14:24:35,721] INFO -   Downloading MarkupSafe-1.1.1-cp37-cp37m-manylinux1_x86_64.whl (27 kB)

[2020-05-07 14:24:35,787] INFO - Collecting ply<4.0,>=3.4

-[2020-05-07 14:24:35,804] INFO -   Downloading ply-3.11-py2.py3-none-any.whl (49 kB)

[2020-05-07 14:24:35,835] INFO - Building wheels for collected packages: fasttext, sqlalchemy-utils, psutil, python-json-logger, cerberus, alembic, prometheus-client, thriftpy2

[2020-05-07 14:24:35,836] INFO -   Building wheel for fasttext (setup.py): started

|[2020-05-07 14:25:28,210] INFO -   Building wheel for fasttext (setup.py): finished with status 'done'

[2020-05-07 14:25:28,222] INFO -   Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=4442198 sha256=9aed31a411e7d482b72bfe

In [21]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name dev-fasttext-classification \
--body '{"text": "Which baking dish is best to bake a banana bread ?"}' --content-type application/json output.json && cat output.json

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "dev-fasttext-cla-FasttextClassification-20200507132536-C23A76"
}
"baking"

In [22]:
!bentoml sagemaker delete fasttext-classification

Successfully deleted AWS Sagemaker deployment "fasttext-classification"
